In [25]:
import ee
import geemap
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import scipy
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import statsmodels.formula.api as sm
import copy
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import sklearn
from math import sqrt
from functools import reduce
import numpy as np
#import ee
#import geemap
import geopandas as gpd
import rioxarray as rxr
import rasterstats as rs
from rasterio.plot import show
import rasterio
import cartopy.crs as ccrs
import gdal
import os
%matplotlib inline
plt.rcParams["font.family"] = "Arial"

In [26]:
Map = geemap.Map(center=(10.9374, 122.6299), zoom=10) # Same as Map.setCenter(122.6299, 10.9374,10)
Map

Map(center=[10.9374, 122.6299], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [27]:
iloilo = ee.FeatureCollection("users/ddel528/ph3adm_iloilo_dissolved")
ph3_adm_bdy = ee.FeatureCollection("users/ddel528/phl_admbnda_adm3_psa_namria_20180130")

In [28]:
Map.addLayer(ph3_adm_bdy, {'color':'green'}, 'ph3_adm_bdy')
Map.addLayer(iloilo, {'color':'red'}, 'iloilo')

# NDVI

In [29]:
# Load MODIS MOD13Q1 dataset.
MOD13Q1 = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI')
MYD13Q1 = ee.ImageCollection('MODIS/006/MYD13Q1').select('NDVI')

# merge collections
imageCollection = MOD13Q1.merge(MYD13Q1).sort("system:time_start").filterBounds(iloilo)

years = ee.List.sequence(2001, 2018)
months = ee.List.sequence(1, 12)

# Monthly Composite
monthlyComposite = ee.ImageCollection.fromImages(
    years.map(lambda y :
              months.map(lambda m :
                         imageCollection.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month'))
                         .max()
                         .divide(10000)
                         .set('system:time_start', ee.Date.fromYMD(y, m, 1).millis())
                         .set('system:index', ee.String.cat('NDVI_', ee.Date.fromYMD(y, m, 1).format('YYYY-MM')))
                        )
             )
    .flatten()
).sort('system:time_start')

decComposite = ee.ImageCollection.fromImages(
  years.map(lambda y:
    monthlyComposite
            .filter(ee.Filter.calendarRange(12, 12, 'month'))
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .sum()
            .set('month','dec')
            .set('system:time_start', ee.Date.fromYMD(y, 12, 1).millis())
            .set('system:index', ee.Date.fromYMD(y, 12, 1).format('YYYY-MM'))
            .set('year',y)
  ))

years = ee.List.sequence(2002, 2019)
augComposite = ee.ImageCollection.fromImages(
  years.map(lambda y:
    monthlyComposite
            .filter(ee.Filter.calendarRange(8, 8, 'month'))
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .sum()
            .set('month','aug')
            .set('system:time_start', ee.Date.fromYMD(y, 8, 1).millis())
            .set('system:index', ee.Date.fromYMD(y, 8, 1).format('YYYY-MM'))
            .set('year',y)
  ))

DecAugColl = decComposite.merge(augComposite).sort('system:time_start')

img2013 = DecAugColl.filter(ee.Filter.date(ee.Date('2012-12-01'), ee.Date('2013-08-30'))).sum().set('system:index', '2013')
img2014 = DecAugColl.filter(ee.Filter.date(ee.Date('2013-12-01'), ee.Date('2014-08-30'))).sum().set('system:index', '2014')
img2015 = DecAugColl.filter(ee.Filter.date(ee.Date('2014-12-01'), ee.Date('2015-08-30'))).sum().set('system:index', '2015')
img2016 = DecAugColl.filter(ee.Filter.date(ee.Date('2015-12-01'), ee.Date('2016-08-30'))).sum().set('system:index', '2016')
img2017 = DecAugColl.filter(ee.Filter.date(ee.Date('2016-12-01'), ee.Date('2017-08-30'))).sum().set('system:index', '2017')
imgList = [img2013, img2014, img2015, img2016, img2017]

ndvi2017 = img2017
ndvi5yr = ee.ImageCollection.fromImages(imgList)
#sumNDVI_5yr.getInfo()
#Map.addLayer(sumNDVI_5yr.mean(), {'palette':['red','yellow','green'],'min':0,'max':3.0}, 'sumNDVI')

# CHIRPS/PCP

In [30]:
# Load MODIS MOD13Q1 dataset.
CHIRPS = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')

# merge collections
imageCollection = CHIRPS.sort("system:time_start").filterBounds(iloilo)

years = ee.List.sequence(2001, 2018)
months = ee.List.sequence(1, 12)

# Monthly Composite
monthlyComposite = ee.ImageCollection.fromImages(
    years.map(lambda y :
              months.map(lambda m :
                         imageCollection.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month'))
                         .sum()
                         .set('system:time_start', ee.Date.fromYMD(y, m, 1).millis())
                         .set('system:index', ee.String.cat('PCP_', ee.Date.fromYMD(y, m, 1).format('YYYY-MM')))
                        )
             )
    .flatten()
).sort('system:time_start')

decComposite = ee.ImageCollection.fromImages(
  years.map(lambda y:
    monthlyComposite
            .filter(ee.Filter.calendarRange(12, 12, 'month'))
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .sum()
            .set('month','dec')
            .set('system:time_start', ee.Date.fromYMD(y, 12, 1).millis())
            .set('system:index', ee.Date.fromYMD(y, 12, 1).format('YYYY-MM'))
            .set('year',y)
  ))

years = ee.List.sequence(2002, 2019)
augComposite = ee.ImageCollection.fromImages(
  years.map(lambda y:
    monthlyComposite
            .filter(ee.Filter.calendarRange(8, 8, 'month'))
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .sum()
            .set('month','aug')
            .set('system:time_start', ee.Date.fromYMD(y, 8, 1).millis())
            .set('system:index', ee.Date.fromYMD(y, 8, 1).format('YYYY-MM'))
            .set('year',y)
  ))

DecAugColl = decComposite.merge(augComposite).sort('system:time_start')

img2013 = DecAugColl.filter(ee.Filter.date(ee.Date('2012-12-01'), ee.Date('2013-08-30'))).sum().set('system:index', '2013')
img2014 = DecAugColl.filter(ee.Filter.date(ee.Date('2013-12-01'), ee.Date('2014-08-30'))).sum().set('system:index', '2014')
img2015 = DecAugColl.filter(ee.Filter.date(ee.Date('2014-12-01'), ee.Date('2015-08-30'))).sum().set('system:index', '2015')
img2016 = DecAugColl.filter(ee.Filter.date(ee.Date('2015-12-01'), ee.Date('2016-08-30'))).sum().set('system:index', '2016')
img2017 = DecAugColl.filter(ee.Filter.date(ee.Date('2016-12-01'), ee.Date('2017-08-30'))).sum().set('system:index', '2017')
imgList = [img2013, img2014, img2015, img2016, img2017]

pcp2017 = img2017
pcp5yr = ee.ImageCollection.fromImages(imgList)

# LST

In [31]:
# Load MODIS LST dataset.
MOD11A1 = ee.ImageCollection('MODIS/006/MOD11A2').select(['LST_Day_1km','LST_Night_1km']);
MYD11A1 = ee.ImageCollection('MODIS/006/MYD11A2').select(['LST_Day_1km','LST_Night_1km']);

# merge collections
imageCollection = MOD11A1.merge(MYD11A1).sort("system:time_start").filterBounds(iloilo)

years = ee.List.sequence(2001, 2018)
months = ee.List.sequence(1, 12)

# Monthly Composite
monthlyComposite = ee.ImageCollection.fromImages(
    years.map(lambda y :
              months.map(lambda m :
                         imageCollection.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month'))
                         .mean()
                         .multiply(0.02)
                         .subtract(273.15)
                         .set('system:time_start', ee.Date.fromYMD(y, m, 1).millis())
                         .set('system:index', ee.String.cat('LST_', ee.Date.fromYMD(y, m, 1).format('YYYY-MM')))
                        )
             )
    .flatten()
).sort('system:time_start')

decComposite = ee.ImageCollection.fromImages(
  years.map(lambda y:
    monthlyComposite
            .filter(ee.Filter.calendarRange(12, 12, 'month'))
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .sum()
            .set('month','dec')
            .set('system:time_start', ee.Date.fromYMD(y, 12, 1).millis())
            .set('system:index', ee.Date.fromYMD(y, 12, 1).format('YYYY-MM'))
            .set('year',y)
  ))

years = ee.List.sequence(2002, 2019)
augComposite = ee.ImageCollection.fromImages(
  years.map(lambda y:
    monthlyComposite
            .filter(ee.Filter.calendarRange(8, 8, 'month'))
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .sum()
            .set('month','aug')
            .set('system:time_start', ee.Date.fromYMD(y, 8, 1).millis())
            .set('system:index', ee.Date.fromYMD(y, 8, 1).format('YYYY-MM'))
            .set('year',y)
  ))

DecAugColl = decComposite.merge(augComposite).sort('system:time_start')

img2013 = DecAugColl.filter(ee.Filter.date(ee.Date('2012-12-01'), ee.Date('2013-08-30'))).mean().set('system:index', '2013')
img2014 = DecAugColl.filter(ee.Filter.date(ee.Date('2013-12-01'), ee.Date('2014-08-30'))).mean().set('system:index', '2014')
img2015 = DecAugColl.filter(ee.Filter.date(ee.Date('2014-12-01'), ee.Date('2015-08-30'))).mean().set('system:index', '2015')
img2016 = DecAugColl.filter(ee.Filter.date(ee.Date('2015-12-01'), ee.Date('2016-08-30'))).mean().set('system:index', '2016')
img2017 = DecAugColl.filter(ee.Filter.date(ee.Date('2016-12-01'), ee.Date('2017-08-30'))).mean().set('system:index', '2017')
imgList = [img2013, img2014, img2015, img2016, img2017]

lst2017 = img2017
lst5yr = ee.ImageCollection.fromImages(imgList)

LST_Day_1km = lst2017.select('LST_Day_1km');
LST_Night_1km = lst2017.select('LST_Night_1km');

# ET

In [32]:
# Load MODIS LST dataset.
MOD16A2 = ee.ImageCollection('MODIS/006/MOD16A2').select(['ET','PET'])

# merge collections
imageCollection = MOD16A2.sort("system:time_start").filterBounds(iloilo)

years = ee.List.sequence(2001, 2018)
months = ee.List.sequence(1, 12)

# Monthly Composite
monthlyComposite = ee.ImageCollection.fromImages(
    years.map(lambda y :
              months.map(lambda m :
                         imageCollection.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month'))
                         .mean() # average evapotranspiration
                         .multiply(0.1)
                         .set('system:time_start', ee.Date.fromYMD(y, m, 1).millis())
                         .set('system:index', ee.String.cat('ET_', ee.Date.fromYMD(y, m, 1).format('YYYY-MM')))
                        )
             )
    .flatten()
).sort('system:time_start')

decComposite = ee.ImageCollection.fromImages(
  years.map(lambda y:
    monthlyComposite
            .filter(ee.Filter.calendarRange(12, 12, 'month'))
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .sum()
            .set('month','dec')
            .set('system:time_start', ee.Date.fromYMD(y, 12, 1).millis())
            .set('system:index', ee.Date.fromYMD(y, 12, 1).format('YYYY-MM'))
            .set('year',y)
  ))

years = ee.List.sequence(2002, 2019)
augComposite = ee.ImageCollection.fromImages(
  years.map(lambda y:
    monthlyComposite
            .filter(ee.Filter.calendarRange(8, 8, 'month'))
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .sum()
            .set('month','aug')
            .set('system:time_start', ee.Date.fromYMD(y, 8, 1).millis())
            .set('system:index', ee.Date.fromYMD(y, 8, 1).format('YYYY-MM'))
            .set('year',y)
  ))

DecAugColl = decComposite.merge(augComposite).sort('system:time_start')

img2013 = DecAugColl.filter(ee.Filter.date(ee.Date('2012-12-01'), ee.Date('2013-08-30'))).mean().set('system:index', '2013')
img2014 = DecAugColl.filter(ee.Filter.date(ee.Date('2013-12-01'), ee.Date('2014-08-30'))).mean().set('system:index', '2014')
img2015 = DecAugColl.filter(ee.Filter.date(ee.Date('2014-12-01'), ee.Date('2015-08-30'))).mean().set('system:index', '2015')
img2016 = DecAugColl.filter(ee.Filter.date(ee.Date('2015-12-01'), ee.Date('2016-08-30'))).mean().set('system:index', '2016')
img2017 = DecAugColl.filter(ee.Filter.date(ee.Date('2016-12-01'), ee.Date('2017-08-30'))).mean().set('system:index', '2017')
imgList = [img2013, img2014, img2015, img2016, img2017]

et2017 = img2017
et5yr = ee.ImageCollection.fromImages(imgList)

et = et2017.select('ET')
pet = et2017.select('PET')

In [33]:
Map.addLayer(et, {'palette':['red','orange','green'],'min':10,'max':35}, 'et2017')

# ADD SHAPEFILE OF TRAINING POINTS

In [34]:
#refPts = geemap.shp_to_ee('/Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/intersect_DS_points_v3.shp')
#in_img = et2017.select('ET').reproject('EPSG:32651')
#geemap.extract_values_to_points(refPts, in_img, 'intersect_test.csv', tile_scale=4)
#out_csv = os.path.join(os.getcwd(), 'intersect_test.csv')
#geemap.extract_values_to_points(refPts, et, out_csv)
#pts.getInfo()
#refPts.getInfo()

In [35]:
out2017 = ee.ImageCollection.fromImages([
    ndvi2017.set({'system:index':'ndvi'}),
    pcp2017.set({'system:index':'pcp'}),
    LST_Day_1km.set({'system:index':'lstday'}),
    LST_Night_1km.set({'system:index':'lstnight'}),
    et.set({'system:index':'et'}),
    pet.set({'system:index':'pet'})])
out2017.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'NDVI',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:index': 'ndvi'}},
  {'type': 'Image',
   'bands': [{'id': 'precipitation',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:index': 'pcp'}},
  {'type': 'Image',
   'bands': [{'id': 'LST_Day_1km',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:index': 'lstday'}},
  {'type': 'Image',
   'bands': [{'id': 'LST_Night_1km',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:index': 'lstnight'}},
  {'type': 'Imag

In [36]:
geemap.ee_export_image_collection(out2017, 'geospatial_layers/out2017', scale=250, crs='EPSG:32651', region=ee.Geometry.dissolve(iloilo))

Total number of images: 6

Exporting 1/6: ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out2017/ndvi.tif


Exporting 2/6: pcp.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out2017/pcp.tif


Exporting 3/6: lstday.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out2017/lstday.tif


Exporting 4/6: lstnight.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out2017/lstnight.tif


Exporting 5/6: et.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/delatorredm/Library/CloudStorage/OneDrive-TheUni

In [38]:
out5yr = ee.ImageCollection.fromImages([
    ndvi5yr.select('NDVI').mean().set({'system:index':'ndvi'}),
    pcp5yr.select('precipitation').mean().set({'system:index':'pcp'}),
    lst5yr.select('LST_Day_1km').mean().set({'system:index':'lstday'}),
    lst5yr.select('LST_Night_1km').mean().set({'system:index':'lstnight'}),
    et5yr.select('ET').mean().set({'system:index':'et'}),
    et5yr.select('PET').mean().set({'system:index':'pet'})])
out5yr.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'features': [{'type': 'Image',
   'bands': [{'id': 'NDVI',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:index': 'ndvi'}},
  {'type': 'Image',
   'bands': [{'id': 'precipitation',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:index': 'pcp'}},
  {'type': 'Image',
   'bands': [{'id': 'LST_Day_1km',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:index': 'lstday'}},
  {'type': 'Image',
   'bands': [{'id': 'LST_Night_1km',
     'data_type': {'type': 'PixelType', 'precision': 'double'},
     'crs': 'EPSG:4326',
     'crs_transform': [1, 0, 0, 0, 1, 0]}],
   'properties': {'system:index': 'lstnight'}},
  {'type': 'Imag

In [ ]:
geemap.ee_export_image_collection(out5yr, 'geospatial_layers/out5yr', scale=250, crs='EPSG:32651', region=ee.Geometry.dissolve(iloilo))

Total number of images: 6

Exporting 1/6: ndvi.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/delatorredm/Library/CloudStorage/OneDrive-TheUniversityofAuckland/Ch4_Climate_Impacts/geospatial_layers/out5yr/ndvi.tif


Exporting 2/6: pcp.tif
Generating URL ...
Please wait ...
